In [1]:
import tensorflow as tf
import argparse
import numpy as np
import sys
sys.path.append("/data/slim/models/research/slim/")
from nets import nets_factory
import matplotlib.pyplot as plt
%matplotlib inline  

sys.path.append("../")
from src.data_preparation import dataset

slim = tf.contrib.slim
PRETAIN_MODEL = "inception_v1"
num_classes = 120
WIDTH = 224
INPUT_WIDTH= 224
CHANNEL = 3

TRAIN_TF_RECORDS = "/data/dog_breeds/stanford_ds/train/train.tfrecord"
TRAIN_BATCH_SIZE = 10

In [2]:
with tf.Graph().as_default(), tf.Session() as sess:
    # First create the dataset and load one batch
    images_raw, images, labels = dataset.load_batch(
        TRAIN_TF_RECORDS,
        TRAIN_BATCH_SIZE,
        INPUT_WIDTH,
        INPUT_WIDTH,
        is_training=True,
        num_epochs=1)
#     images = tf.cast(images * 255, tf.uint8)
    sess.run(tf.local_variables_initializer())
    
    with slim.queues.QueueRunners(sess):
        image_raw_batch, image_batch = sess.run([images_raw, images])
        print(image_batch.max(), image_batch.min())

#     print(image_batch[0])   
    image_precessed_batch = (image_batch * 127 + 127).astype(np.uint8)
#     print(image_precessed_batch[0])
    print(image_precessed_batch.max(), image_precessed_batch.min())
    plt.imshow(image_precessed_batch[0])
    plt.show()

    plt.imshow(image_raw_batch[0, :].astype(np.uint8))
    plt.show()


ValueError: setting an array element with a sequence.

In [4]:
with tf.Graph().as_default(), tf.Session() as sess:
    x_input = tf.placeholder(tf.float32, [None, WIDTH, HEIGHT, CHANNEL])
    label = tf.placeholder(tf.int64)
    
    net_fn = nets_factory.get_network_fn(
            PRETAIN_MODEL,
            num_classes,
            is_training=False)
    
    logits, end_points = net_fn(x_input)
    predictions = end_points['Predictions']
    
    sess.run(tf.global_variables_initializer())

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
    
    x = np.ones([1, HEIGHT, WIDTH, CHANNEL])
    loss_, logits_ = sess.run([loss, logits], {x_input: x, label: [1]})
    print("{}, {}".format(loss_, logits_))

INFO:tensorflow:Scale of 0 disables regularizer.
[4.787492], [[-5.32276825e-14  1.64103621e-13 -7.32893455e-13  6.89111854e-13
   1.12389292e-13  1.44264768e-13 -1.78983822e-13 -7.18660049e-13
   6.08263914e-14  6.99800081e-13 -1.90176922e-13 -4.03928261e-14
  -2.66548556e-13  6.79433452e-14 -8.52541074e-13 -7.89342753e-14
  -1.41762375e-13  4.97536636e-13 -1.53853140e-13  4.76851496e-14
  -2.65714439e-14  1.76175020e-13 -1.09750926e-13 -5.15103436e-14
   8.16989705e-13  1.61399621e-13  2.06371758e-13 -1.54762501e-13
  -4.94505316e-13  5.41010379e-13  1.20434993e-13  1.96137730e-13
   1.31995630e-12  5.86474025e-14  2.56242591e-13 -6.64955775e-13
   4.17993846e-13  2.32445329e-13  1.87418278e-13  4.63848794e-16
  -5.19684176e-13 -3.54817819e-13  1.72257906e-13  2.00170067e-13
   9.92542298e-14 -1.42187423e-13 -7.00591278e-13  3.79204860e-14
  -9.12988435e-13 -2.02091084e-13 -4.35360976e-13  6.49601832e-14
   2.85671552e-13  4.60904101e-14  4.21625625e-13  4.59204940e-13
  -2.72722003e-